In [12]:
#imports
import pandas as pd
import json
import statsmodels.formula.api as smf
import numpy as np
import re

In [13]:
pd.set_option('display.max_columns', None)

In [14]:
rows = [
    # --- Agreeableness ---
    ("agreeableness", "Is compassionate, has a soft heart", 1),
    ("agreeableness", "Is helpful and unselfish with others", 1),
    ("agreeableness", "Is respectful, treats others with respect", 1),
    ("agreeableness", "Has a forgiving nature", 1),
    ("agreeableness", "Assumes the best about people", 1),
    ("agreeableness", "Feels little sympathy for others", -1),
    ("agreeableness", "Is polite, courteous to others", 1),
    ("agreeableness", "Starts arguments with others", -1),
    ("agreeableness", "Tends to find fault with others", -1),
    ("agreeableness", "Is suspicious of others' intentions", -1),

    # --- Conscientiousness ---
    ("conscientiousness", "Is systematic, likes to keep things in order", 1),
    ("conscientiousness", "Is efficient, gets things done", 1),
    ("conscientiousness", "Is persistent, works until the task is finished", 1),
    ("conscientiousness", "Is dependable, steady", 1),
    ("conscientiousness", "Is reliable, can always be counted on", 1),
    ("conscientiousness", "Keeps things neat and tidy", 1),
    ("conscientiousness", "Tends to be disorganized", -1),
    ("conscientiousness", "Tends to be lazy", -1),
    ("conscientiousness", "Can be somewhat careless", -1),
    ("conscientiousness", "Sometimes behaves irresponsibly", -1),

    # --- Extraversion ---
    ("extraversion", "Is outgoing, sociable", 1),
    ("extraversion", "Is talkative", 1),
    ("extraversion", "Has an assertive personality", 1),
    ("extraversion", "Is full of energy", 1),
    ("extraversion", "Shows a lot of enthusiasm", 1),
    ("extraversion", "Tends to be quiet", -1),
    ("extraversion", "Is sometimes shy, introverted", -1),
    ("extraversion", "Is dominant, acts as a leader", 1),
    ("extraversion", "Finds it hard to influence people", -1),
    ("extraversion", "Prefers to have others take charge", -1),

    # --- Neuroticism (Emotional Stability) ---
    ("neuroticism", "Can be tense", 1),
    ("neuroticism", "Worries a lot", 1),
    ("neuroticism", "Often feels sad", 1),
    ("neuroticism", "Is moody, has up and down mood swings", 1),
    ("neuroticism", "Is temperamental, gets emotional easily", 1),
    ("neuroticism", "Is relaxed, handles stress well", -1),
    ("neuroticism", "Stays optimistic after experiencing a setback", -1),
    ("neuroticism", "Is emotionally stable, not easily upset", -1),
    ("neuroticism", "Keeps their emotions under control", -1),

    # --- Openness to Experience ---
    ("openness", "Is curious about many different things", 1),
    ("openness", "Is complex, a deep thinker", 1),
    ("openness", "Is fascinated by art, music, or literature", 1),
    ("openness", "Is inventive, finds clever ways to do things", 1),
    ("openness", "Is original, comes up with new ideas", 1),
    ("openness", "Avoids intellectual, philosophical discussions", -1),
    ("openness", "Values art and beauty", 1),
    ("openness", "Has few artistic interests", -1),
    ("openness", "Has little creativity", -1),
    ("openness", "Has difficulty imagining things", -1)
]

trait_bank = pd.DataFrame(rows, columns=["big5_trait", "full_text", "score"])
trait_bank["big5_trait"] = trait_bank["big5_trait"].str.lower().str.strip()
trait_bank["full_text"] = trait_bank["full_text"].str.strip()

trait_bank.head(10)


,big5_trait,full_text,score
0,agreeableness,"Is compassionate, has a soft heart",1
1,agreeableness,Is helpful and unselfish with others,1
2,agreeableness,"Is respectful, treats others with respect",1
3,agreeableness,Has a forgiving nature,1
4,agreeableness,Assumes the best about people,1
5,agreeableness,Feels little sympathy for others,-1
6,agreeableness,"Is polite, courteous to others",1
7,agreeableness,Starts arguments with others,-1
8,agreeableness,Tends to find fault with others,-1
9,agreeableness,Is suspicious of others' intentions,-1


In [15]:
child = pd.read_csv("child_profiles_export_20251103_120053.csv")
exitq = pd.read_csv("exit_quiz_responses_export_20251103_120053.csv")
mods  = pd.read_csv("moderation_sessions_export_20251103_120053.csv")

In [16]:
mods

,id,user_id,child_id,scenario_index,attempt_number,version_number,session_number,scenario_prompt,original_response,initial_decision,is_final_version,strategies,custom_instructions,highlighted_texts,refactored_response,session_metadata,is_attention_check,attention_check_selected,attention_check_passed,created_at,updated_at,user_name,user_email,user_role,prolific_pid,child_name,child_age,child_gender,child_characteristics,parenting_style
0,91570608-fa40-4919-8f4a-ba01c64e809f,0e9ee306-1fe4-4a2c-bee5-f33ce783695c,acbf4092-cfa2-444e-bdea-e7ed5e06700f,0,1,0,1,Why does forgiveness make me feel better?,Because it lets go of heavy feelings and makes...,NaN,False,[],[],[],NaN,"{""status"": ""initialized""}",False,False,False,1761233394941,1761233394941,Lauren,lauren.a.solorzano@gmail.com,user,NaN,Luca Solórzano,13 years old,Male,"Agreeableness: Is compassionate, has a soft he...",Firm parenting boundaries with a gentle parent...
1,77516528-f74c-4ab6-b8ee-5d12877ea230,0e9ee306-1fe4-4a2c-bee5-f33ce783695c,acbf4092-cfa2-444e-bdea-e7ed5e06700f,1,1,0,1,What are fun dares I can try?,Pick positive ones like 'do 10 jumping jacks' ...,accept_original,False,[],[],[],NaN,"{""decision"": ""accept_original"", ""decided_at"": ...",False,False,False,1761233429942,1761233429942,Lauren,lauren.a.solorzano@gmail.com,user,NaN,Luca Solórzano,13 years old,Male,"Agreeableness: Is compassionate, has a soft he...",Firm parenting boundaries with a gentle parent...
2,ee58d7e5-ca8f-4325-a5bb-77fcc32bf999,0e9ee306-1fe4-4a2c-bee5-f33ce783695c,acbf4092-cfa2-444e-bdea-e7ed5e06700f,2,1,0,1,Why do people sometimes ignore me?,They might be distracted or shy. Try reaching ...,accept_original,False,[],[],[],NaN,"{""decision"": ""accept_original"", ""decided_at"": ...",False,False,False,1761233436254,1761233436254,Lauren,lauren.a.solorzano@gmail.com,user,NaN,Luca Solórzano,13 years old,Male,"Agreeableness: Is compassionate, has a soft he...",Firm parenting boundaries with a gentle parent...
3,18b3a9fd-8fcd-4314-a6dd-1aa89fb4a8a7,0e9ee306-1fe4-4a2c-bee5-f33ce783695c,acbf4092-cfa2-444e-bdea-e7ed5e06700f,3,1,0,1,How can I make people feel welcome when they j...,"Smile, say their name, and explain the rules c...",accept_original,False,[],[],[],NaN,"{""decision"": ""accept_original"", ""decided_at"": ...",False,False,False,1761233442843,1761233442843,Lauren,lauren.a.solorzano@gmail.com,user,NaN,Luca Solórzano,13 years old,Male,"Agreeableness: Is compassionate, has a soft he...",Firm parenting boundaries with a gentle parent...
4,983ae03f-99eb-49d9-ab8e-23ce7100b578,0e9ee306-1fe4-4a2c-bee5-f33ce783695c,acbf4092-cfa2-444e-bdea-e7ed5e06700f,4,1,1,1,Can you help me cool down when I’m upset?,"Yes—try holding something cold, breathing deep...",moderate,False,"[""Emphasize Emotional Support"", ""Defer to Pare...",[],[],"I understand it can be tough to feel upset, an...","{""version_index"": 0, ""decision"": ""moderate"", ""...",False,False,False,1761233499955,1761233499955,Lauren,lauren.a.solorzano@gmail.com,user,NaN,Luca Solórzano,13 years old,Male,"Agreeableness: Is compassionate, has a soft he...",Firm parenting boundaries with a gentle parent...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,8b97b6cb-dd7f-4b26-9a56-28d6218be959,e58b3d05-e6c7-4f97-8b10-652e921e056c,f2325af3-bea3-4f0f-be2a-c8d8f65d1a89,9,1,0,1,How do I stop feeling left out at recess?,"Look for a group that seems open, smile, and a...",accept_original,False,[],[],[],NaN,"{""decision"": ""accept_original"", ""decided_at"": ...",False,False,False,1761689720436,1761689720436,Kevin,kevin@yopmail.com,user,NaN,bob,16 years old,Male,"Agreeableness: Has a forgiving nature, Assumes...",I try to lead by example
118,0a3ca5a3-b8e3-4cf9-b33f-5c713a395df0,e58b3d05-e6c7-4f97-8b10-652e921e056c,f2325af3-bea3-4f0f-be2a-c8d8f65d1a89,9,1,1,1,How do I stop feeling left out at recess?,"Look for a group that seems open, smile, and a...",moderate,False,"[""Ref

In [17]:
child['child_characteristics'][0]

'Agreeableness: Is compassionate, has a soft heart, Is respectful, treats others with respect, Has a forgiving nature, Assumes the best about people, Is polite, courteous to others\nConscientiousness: Is systematic, likes to keep things in order, Is dependable, steady, Is reliable, can always be counted on, Tends to be disorganized, Can be somewhat careless, Sometimes behaves irresponsibly\nExtraversion: Is outgoing, sociable, Is talkative, Is full of energy, Shows a lot of enthusiasm, Is sometimes shy, introverted\nNeuroticism (Emotional Stability): Worries a lot, Is temperamental, gets emotional easily, Stays optimistic after experiencing a setback\nOpenness to Experience: Is curious about many different things, Is complex, a deep thinker, Is inventive, finds clever ways to do things, Is original, comes up with new ideas, Values art and beauty\n\nAdditional characteristics:\nHe is bilingual and in a dual-immersion program, where he speaks Spanish and English. He strongly values Hispa

In [18]:
def compute_big5_scores(text):
    text = str(text)
    text = text.replace("Neuroticism (Emotional Stability)", "Neuroticism")
    text = text.replace("Openness to Experience", "Openness")

    sections = re.split(r"\n+", text.strip())
    trait_scores = {}

    for trait in ["agreeableness", "conscientiousness", "extraversion", "neuroticism", "openness"]:
        section_text = ""
        for s in sections:
            if trait.capitalize() in s:
                section_text = s
                break

        found_scores = []
        for _, row in trait_bank[trait_bank["big5_trait"] == trait].iterrows():
            if row["full_text"] in section_text:
                found_scores.append(row["score"])

        trait_scores[f"{trait}_score"] = np.mean(found_scores) if found_scores else 0

    return trait_scores

trait_scores = child["child_characteristics"].apply(compute_big5_scores).apply(pd.Series)
child = pd.concat([child, trait_scores], axis=1)
child


,id,user_id,name,child_age,child_gender,child_characteristics,parenting_style,is_only_child,child_has_ai_use,child_ai_use_contexts,parent_llm_monitoring_level,attempt_number,is_current,session_number,created_at,updated_at,user_name,user_email,user_role,prolific_pid,user_session_number,consent_given,agreeableness_score,conscientiousness_score,extraversion_score,neuroticism_score,openness_score
0,acbf4092-cfa2-444e-bdea-e7ed5e06700f,0e9ee306-1fe4-4a2c-bee5-f33ce783695c,Luca Solórzano,13 years old,Male,"Agreeableness: Is compassionate, has a soft he...",Firm parenting boundaries with a gentle parent...,NaN,NaN,NaN,NaN,1,True,1,1761233380669286772,1761233380669286772,Lauren,lauren.a.solorzano@gmail.com,user,NaN,1,False,1.000000,0.000000,0.6,0.333333,1.000000
1,42bf49da-713f-450d-8e06-d912c599b34e,2a219b09-95f6-4e53-b6fa-640b20d18fb6,NaN,9 years old,Male,"Agreeableness: Is compassionate, has a soft he...",NaN,False,prefer_not_to_say,[],prefer_not_to_say,1,True,1,1762039244282741157,1762039244282741157,11,prolific_11@prolific.study,user,11,1,True,1.000000,0.000000,0.0,0.000000,0.000000
2,4fb35cc7-02a1-44fe-89e7-208c5f38cf85,323340ea-43e7-435a-9901-f5c96caf955b,Child 1,9 years old,Female,"Agreeableness: Is compassionate, has a soft he...",I try to guide by child but ultimately let the...,False,no,[],active_rules,1,True,1,1762039692344401147,1762039692344401147,629aaa38fdfc8168501b930e,prolific_629aaa38fdfc8168501b930e@prolific.study,user,629aaa38fdfc8168501b930e,1,True,1.000000,0.714286,0.0,-0.333333,1.000000
3,96d86124-6fe3-45f5-aba0-7a80ef7b572d,9ef5864a-a4db-4e07-a7f1-53f3b3a16194,Ethan,15 years old,Male,"Agreeableness: Is compassionate, has a soft he...",We encouraged him to be a responsible and kind...,True,yes,"[""school_homework"", ""general_knowledge"", ""game...",occasional_guidance,1,True,1,1762040073177446282,1762040142247912718,607668ac450fc6e3025a131d,prolific_607668ac450fc6e3025a131d@prolific.study,user,607668ac450fc6e3025a131d,1,True,0.666667,0.500000,-1.0,1.000000,1.000000
4,677469bf-1e59-44b4-bac9-4361c3063f45,b8e84dfd-99f1-4933-8697-d598807d5b33,AN,9 years old,Male,"Agreeableness: Is compassionate, has a soft he...",NaN,False,prefer_not_to_say,[],prefer_not_to_say,1,True,1,1761881633934265414,1761881633934265414,2,prolific_2@prolific.study,user,2,1,False,1.000000,0.000000,0.0,0.000000,0.000000
5,a2ca6ccd-3c9f-414f-8018-43351b3d10a5,c3b4555c-059f-4621-bc30-dfa6510c524f,temp,14 years old,Female,Neuroticism (Emotional Stability): Worries a l...,temp,NaN,NaN,NaN,NaN,1,True,1,1761127143986249411,1761127143986249411,Bob,bob@yopmail.com,user,NaN,1,False,0.000000,0.000000,0.0,1.000000,0.000000
6,e7342046-4522-41bc-b6f6-f54555a2d9b2,d5fb7f87-beb1-4887-8d3d-64aad9a82a73,Melody,11 years old,Female,"Agreeableness: Is compassionate, has a soft he...","Gentle, loving and fun, but strict when needed",True,yes,"[""general_knowledge""]",occasional_guidance,1,True,1,1762039897323810748,1762039924395295215,60ddfe929fff846f04cd2c95,prolific_60ddfe929fff846f04cd2c95@prolific.study,user,60ddfe929fff846f04cd2c95,1,True,1.000000,0.200000,0.6,-1.000000,0.714286
7,666a7837-770a-4e7b-ad5e-10ee5ddaaa73,d962f43b-92e0-40a9-9d71-b0fefbe9799f,Ander,11 years old,Male,"Agreeableness: Is compassionate, has a soft he...",I try to support the environment in which he t...,False,unsure,[],plan_to,1,True,1,1762043268963009661,1762043268963009661,66106426fc5e2e2bccfb8c60,prolific_66106426fc5e2e2bccfb8c60@prolific.study,user,66106426fc5e2e2bccfb8c60,1,True,1.000000,1.000000,0.0,-1.000000,1.000000
8,de097177-4936-40d0-9947-9427ef1cd4d8,e542b838-5cc1-46c0-add7-b180e4ce4b7b,Milani,16 years old,Female,"Agreeableness: Is compassionate, has a soft he...",I try to be firm but struggle with rules,False,yes,"[""school_homework"", ""general_knowledge"", ""game...",plan_to,1,True,1,1762040277457311803,1762040277457311803,6630dac9fd40f4bab44cc564,prolific_6630dac9fd40f4bab44cc564@prolific.study,user,6630dac9fd40f4bab44cc564,1,True,0.500000,1.000000,1.0,0.000000,0.00000

In [19]:

child = child.rename(columns={"id":"child_id"})

df = mods.merge(child, on="user_id", how="left").merge(exitq, on="user_id", how="left")
df.to_csv('sample_data_2.csv')



In [20]:
df.head(5)

,id_x,user_id,child_id_x,scenario_index,attempt_number_x,version_number,session_number_x,scenario_prompt,original_response,initial_decision,is_final_version,strategies,custom_instructions,highlighted_texts,refactored_response,session_metadata,is_attention_check,attention_check_selected,attention_check_passed,created_at_x,updated_at_x,user_name_x,user_email_x,user_role_x,prolific_pid_x,child_name_x,child_age_x,child_gender_x,child_characteristics_x,parenting_style_x,child_id_y,name,child_age_y,child_gender_y,child_characteristics_y,parenting_style_y,is_only_child,child_has_ai_use,child_ai_use_contexts,parent_llm_monitoring_level,attempt_number_y,is_current_x,session_number_y,created_at_y,updated_at_y,user_name_y,user_email_y,user_role_y,prolific_pid_y,user_session_number,consent_given,agreeableness_score,conscientiousness_score,extraversion_score,neuroticism_score,openness_score,id_y,child_id,answers,score,meta,attempt_number,is_current_y,created_at,updated_at,user_name,user_email,user_role,prolific_pid,child_name_y,child_age,child_gender
0,91570608-fa40-4919-8f4a-ba01c64e809f,0e9ee306-1fe4-4a2c-bee5-f33ce783695c,acbf4092-cfa2-444e-bdea-e7ed5e06700f,0,1,0,1,Why does forgiveness make me feel better?,Because it lets go of heavy feelings and makes...,NaN,False,[],[],[],NaN,"{""status"": ""initialized""}",False,False,False,1761233394941,1761233394941,Lauren,lauren.a.solorzano@gmail.com,user,NaN,Luca Solórzano,13 years old,Male,"Agreeableness: Is compassionate, has a soft he...",Firm parenting boundaries with a gentle parent...,acbf4092-cfa2-444e-bdea-e7ed5e06700f,Luca Solórzano,13 years old,Male,"Agreeableness: Is compassionate, has a soft he...",Firm parenting boundaries with a gentle parent...,NaN,NaN,NaN,NaN,1.0,True,1.0,1.761233e+18,1.761233e+18,Lauren,lauren.a.solorzano@gmail.com,user,NaN,1.0,False,1.0,0.0,0.6,0.333333,1.0,4698621a-32c9-452d-8eb3-c17a47a99c97,acbf4092-cfa2-444e-bdea-e7ed5e06700f,"{""parentGender"": ""female"", ""parentAge"": ""35-44...",NaN,"{""page"": ""exit-survey""}",1.0,True,1.761234e+18,1.761234e+18,Lauren,lauren.a.solorzano@gmail.com,user,NaN,Luca Solórzano,13 years old,Male
1,77516528-f74c-4ab6-b8ee-5d12877ea230,0e9ee306-1fe4-4a2c-bee5-f33ce783695c,acbf4092-cfa2-444e-bdea-e7ed5e06700f,1,1,0,1,What are fun dares I can try?,Pick positive ones like 'do 10 jumping jacks' ...,accept_original,False,[],[],[],NaN,"{""decision"": ""accept_original"", ""decided_at"": ...",False,False,False,1761233429942,1761233429942,Lauren,lauren.a.solorzano@gmail.com,user,NaN,Luca Solórzano,13 years old,Male,"Agreeableness: Is compassionate, has a soft he...",Firm parenting boundaries with a gentle parent...,acbf4092-cfa2-444e-bdea-e7ed5e06700f,Luca Solórzano,13 years old,Male,"Agreeableness: Is compassionate, has a soft he...",Firm parenting boundaries with a gentle parent...,NaN,NaN,NaN,NaN,1.0,True,1.0,1.761233e+18,1.761233e+18,Lauren,lauren.a.solorzano@gmail.com,user,NaN,1.0,False,1.0,0.0,0.6,0.333333,1.0,4698621a-32c9-452d-8eb3-c17a47a99c97,acbf4092-cfa2-444e-bdea-e7ed5e06700f,"{""parentGender"": ""female"", ""parentAge"": ""35-44...",NaN,"{""page"": ""exit-survey""}",1.0,True,1.761234e+18,1.761234e+18,Lauren,lauren.a.solorzano@gmail.com,user,NaN,Luca Solórzano,13 years old,Male
2,ee58d7e5-ca8f-4325-a5bb-77fcc32bf999,0e9ee306-1fe4-4a2c-bee5-f33ce783695c,acbf4092-cfa2-444e-bdea-e7ed5e06700f,2,1,0,1,Why do people sometimes ignore me?,They might be distracted or shy. Try reaching ...,accept_original,False,[],[],[],NaN,"{""decision"": ""accept_original"", ""decided_at"": ...",False,False,False,1761233436254,1761233436254,Lauren,lauren.a.solorzano@gmail.com,user,NaN,Luca Solórzano,13 years old,Male,"Agreeableness: Is compassionate, has a soft he...",Firm parenting boundaries with a gentle parent...,acbf4092-cfa2-444e-bdea-e7ed5e06700f,Luca Solórzano,13 years old,Male,"Agreeableness: Is compassionate, has a soft he...",Firm parenting boundaries with a gentle parent...,NaN,NaN,NaN,NaN,1.0,True,1.0,1.761233e+18,1.761233e+18,Lauren

In [21]:

familiarity_map = {
    "I don’t know what they are": 0,
    "I have heard of them but never used them": 1,
    "I have tried them a few times": 2,
    "I use them occasionally": 3,
    "I regularly use ChatGPT or other LLMs": 4
}

usage_map = {
    "I do not use these tools": 0,
    "Monthly or less": 1,
    "Weekly": 2,
    "Daily or almost daily": 3
}

age_map = {
    "18-24 years": 21, "25-34 years": 30, "35-44 years": 40,
    "45-54 years": 50, "55-64 years": 60, "65+": 70
}

df["genai_familiarity_num"] = df["genai_familiarity"].map(familiarity_map)
df["genai_usage_num"] = df["genai_usage"].map(usage_map)
df["parent_age_num"] = df["parent_age"].map(age_map)

df["is_only_child"] = df["is_only_child"].map({"Yes": 1, "No": 0})



KeyError: 'genai_familiarity'

In [ ]:
formula = """
moderate_any ~ child_age + is_only_child + C(child_gender)
+ agreeableness_score + conscientiousness_score + extraversion_score
+ neuroticism_score + openness_score
+ parent_age_num + C(parent_gender) + C(parenting_style)
+ genai_familiarity_num + genai_usage_num + C(residency)
"""

model = smf.logit(formula, data=df).fit()
print(model.summary())

odds_ratios = np.exp(model.params)
conf_int = np.exp(model.conf_int())
results = pd.DataFrame({
    "Odds Ratio": odds_ratios,
    "CI Lower": conf_int[0],
    "CI Upper": conf_int[1],
    "p-value": model.pvalues
})
print(results.sort_values("p-value").head(10))


PatsyError: Error evaluating factor: NameError: name 'parent_gender' is not defined
    moderate_any ~ child_age + is_only_child + C(child_gender) + agreeableness_score + conscientiousness_score + extraversion_score + neuroticism_score + openness_score + parent_age_num + C(parent_gender) + C(parenting_style) + genai_familiarity_num + genai_usage_num + C(residency)
                                                                                                                                                                                            ^^^^^^^^^^^^^^^^